# Raven Pharmacy
Extract all Raven pharmacy records for `coh_pt` between date parameters

**Script**
* [scripts/de/raven_pharmacy.ipynb](./scripts/de/raven_pharmacy.ipynb)

**Prior Script(s)**
* [scripts/coh/coh_basic.ipynb](./scripts/coh/coh_basic.ipynb)

**Parameters**
* `in/de/raven_extract.xlsx[raven_extract]`

**Input**
* `coh_pt`
* `rwd_db.rwd.raven_external_pharmacy`

**Output**  
* `de_raven_pharmacy`

**Review**
* [scripts/de/raven_pharmacy.html](./scripts/de/raven_pharmacy.html)

In [ ]:
#Import libraries for this notebook
import pandas as pd  
from drg_connect import Snowflake
import numpy as np
import pickle
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Load connection variables to connect_dict
with open('../../out/conn/connect_dict.pickle', 'rb') as handle:
    connect_dict = pickle.load(handle)

#Create Eegine to connect to snowflake
snow = Snowflake(role=connect_dict['role'],
                 warehouse=connect_dict['warehouse'],
                 database=connect_dict['database'],
                 schema=connect_dict['schema'])

#Finish engine setup
engine = snow.engine
%load_ext sql_magic
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications


# Parameters
Create python variables of the parameters

 **Input**  
* `in/extract/raven_extract.xlsx[raven_extract]`

**Output**
* Python variables named after parameters with the value

In [ ]:
#Create system variables from excel into script and review values in dictionary
input_df = pd.read_excel('../../in/de/raven_extract.xlsx', sheet_name='raven_extract', skiprows=4, dtype=str)
var_dict = dict(zip(input_df.parameter, input_df.value))
for key,val in var_dict.items(): exec(key + '=val')

#Check inputs
pd.DataFrame.from_dict(var_dict, orient='index')

# Extract Data
Extract subset of raven diagnosis for the patients of interest between specified date ranges

**Parameters**
  * `pharmacy_start_dt`
  * `pharmacy_end_dt`  
  
**Input**
  * `coh_pt`
  * `rwd_db.rwd.raven_external_pharmacy`
  
**Output**  
* `de_raven_pharmacy`

In [ ]:
%%read_sql
--Create Raven pharmacy table
DROP TABLE IF EXISTS de_raven_pharmacy;
CREATE TRANSIENT TABLE de_raven_pharmacy AS
    SELECT ( patient_id 
             || prescription_or_service_reference_number 
             || fill_number 
             || Trim(product_or_service_id) ) AS script_id, 
           patient_id, 
           claim_id, 
           prescriber_id                      AS npi, 
           product_or_service_id              AS ndc, 
           date_of_service, 
           Year(date_of_service)              AS service_year, 
           Quarter(date_of_service)           AS service_quarter, 
           zip3, 
           prescription_or_service_reference_number, 
           ncpdp_number, 
           date_prescription_written, 
           gross_amount_due_submitted, 
           patient_paid_amount_submitted, 
           patient_pay_amount, 
           total_amount_paid, 
           type_of_payment, 
           days_supply, 
           fill_number, 
           quantity_dispensed, 
           number_of_refills_authorized, 
           transaction_code, 
           response_code, 
           reject_code_1, 
           reject_code_2, 
           reject_code_3, 
           reject_code_4, 
           reject_code_5, 
           provider_id, 
           provider_npi, 
           plan_name, 
           date_authorized, 
           time_authorized, 
           etl_update_ts, 
           data_source 
      #FROM rwd_db.rwd.raven_pharmacy 
      FROM rwd_db.rwd.raven_external_pharmacy
     WHERE (date_of_service BETWEEN '{pharmacy_start_dt}' AND '{pharmacy_end_dt}' )
           AND (patient_id IN (SELECT patient_id 
                                FROM coh_pt)
               OR claim_id IN (SELECT claim_id
                                 FROM coh_basic_raven_pharmacy)); 

In [ ]:
%%read_sql
--Review counts as a sanity check
SELECT Count(*) AS row_cnt,
       Count(Distinct patient_id) AS patient_cnt
  FROM de_raven_pharmacy;